# STEP 2
## Data Exploration
## Featurization

In [1]:
import sqlite3
import csv
import pandas as pd

In [2]:
conn = sqlite3.connect('sabs_moonshot.db')

# Extract the compounds with missing descriptors

In [8]:
df = pd.read_sql_query("""
SELECT 
    compounds.id,
    compounds.MW,
    rdkit_descriptors.HBA,
    rdkit_descriptors.HBD, rdkit_descriptors.cLogP,
    compounds.smiles
FROM rdkit_descriptors 
INNER JOIN compounds on compounds.id = rdkit_descriptors.compound_id
WHERE HBA = "" OR HBD = "" OR MW = "" OR cLogP = "" AND assayed = TRUE;
""",
conn)
print(df)

                     id MW HBA HBD cLogP  \
0    DAR-DIA-6a508060-1                    
1    DAR-DIA-6a508060-2                    
2    DAR-DIA-6a508060-3                    
3    DAR-DIA-6a508060-4                    
4    DAR-DIA-6a508060-5                    
5    DAR-DIA-6a508060-7                    
6    DAR-DIA-6a508060-8                    
7    DAR-DIA-6a508060-9                    
8   DAR-DIA-6a508060-10                    
9   DAR-DIA-6a508060-11                    
10  DAR-DIA-6a508060-12                    
11  DAR-DIA-6a508060-13                    
12  DAR-DIA-6a508060-14                    
13  DAR-DIA-6a508060-15                    
14  DAR-DIA-6a508060-16                    
15   ALP-UNI-b33a865d-1                    
16   ALP-UNI-b33a865d-1                    
17   ALP-UNI-b33a865d-1                    
18   DAR-DIA-6a508060-3                    
19   DAR-DIA-6a508060-4                    
20   PRA-UNK-2c426785-1                    
21   EDG-MED-0c930815-1         